In [7]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
import numpy as np
from sklearn.metrics import mean_squared_error
import mlflow
import mlflow.xgboost
import xgboost as xgb
from sklearn.metrics import mean_absolute_error

In [2]:
data = pd.read_csv(r'flats_to_buy_preproceesed.csv')


data.dropna(subset=['ConstructionYear'], inplace=True)
data.dropna(subset=['Object_price'], inplace=True)
data.dropna(subset=['Rooms'], inplace=True)
data.drop('Title', axis=1, inplace=True)
data.drop('Object_currency', axis=1, inplace=True)
data.drop('Url', axis=1, inplace=True)
data.drop('EstateType', axis=1, inplace=True)
data.drop('DistributionType', axis=1, inplace=True)
data = data.reindex()
data = data.reset_index(drop=True)

y = data['Object_price']
X = data.drop('Object_price', axis=1)

In [3]:
train_X, test_X, train_y, test_y = train_test_split(X, y, 
                                                  train_size=0.8, 
                                                  random_state = 0)
train_X, val_X, train_y, val_y = train_test_split(train_X, train_y, 
                                                  train_size=0.8, 
                                                  random_state = 0)

num_cols = [col for col in train_X.columns if train_X[col].dtype == 'float64']
cat_cols = [col for col in train_X.columns if train_X[col].dtype == 'object']
bin_cols = [col for col in train_X.columns if train_X[col].dtype == 'int64']

In [4]:
encoder = OneHotEncoder(handle_unknown='ignore')
X_train_categorical_xgb = encoder.fit_transform(train_X[cat_cols]).toarray()
X_valid_categorical_xgb = encoder.transform(val_X[cat_cols]).toarray()
X_test_categorical_xgb = encoder.transform(test_X[cat_cols]).toarray()
X_train_processed = np.concatenate([train_X[num_cols], X_train_categorical_xgb], axis=1)
X_valid_processed = np.concatenate([val_X[num_cols], X_valid_categorical_xgb], axis=1)
X_test_processed = np.concatenate([test_X[num_cols], X_test_categorical_xgb], axis=1)
X_train_processed = np.concatenate([train_X[bin_cols], X_train_processed], axis=1)
X_valid_processed = np.concatenate([val_X[bin_cols], X_valid_processed], axis=1)
X_test_processed = np.concatenate([test_X[bin_cols], X_test_processed], axis=1)

In [5]:
mlflow.set_tracking_uri("http://localhost:5000")

NameError: name 'mlflow' is not defined

In [6]:
mlflow.end_run()


NameError: name 'mlflow' is not defined

In [ ]:


mlflow.xgboost.autolog()

# Start an MLflow run
mlflow.start_run()


model = xgb.XGBRegressor(eval_metric=['rmse', 'mae'], early_stopping_rounds=59)
model.fit(X=X_train_processed, 
          y=train_y,
          eval_set=[(X_valid_processed, val_y)],
          verbose=True)

mlflow.xgboost.log_model(model, "xgboost-model")

preds = model.predict(X_test_processed)
score = mean_absolute_error(test_y, preds)

mlflow.end_run()


2023/07/05 18:05:17 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of xgboost. If you encounter errors during autologging, try upgrading / downgrading xgboost to a supported version, or try upgrading MLflow.


Exception: Run with UUID b2ea8904bd0941e6865ebaabbcf205b4 is already active. To start a new run, first end the current run with mlflow.end_run(). To start a nested run, call start_run with nested=True